In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
from art.defences.preprocessor.gaussian_augmentation import GaussianAugmentation

gaussian_augmenter = GaussianAugmentation()

def preprocess(X,y, gaussian_augmenter):
    x_test, y_test = gaussian_augmenter(X, y)

    return torch.tensor(x_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long)


In [3]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',

        }
head = pd.DataFrame([head])
head.to_csv("./gaussian_augmenter.csv", mode='a', index=False)




In [4]:
def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_test))
    
    # X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    X_test_tensor , y_test_tensor= preprocess(X_test, y_test, gaussian_augmenter)
    X_test_tensor = X_test_tensor[:len(X_test)] 
    y_test_tensor = y_test_tensor[:len(y_test)] 
    X_test_tensor = X_test_tensor.to(device)
    y_test_tensor = y_test_tensor.to(device)

    # X_test_tensor = preprocess(X_test, gaussian_augmenter).to(device)
    # y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)
    
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    
        

        print(f"Accuracy: {accuracy}")
        
        print("\nmacro")
        print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
        print("\nweighted")
        print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
        print()
        


        new_row = {
            "model" : model_name,
            "attack_model" : attack_name,
            'epsilon': eps,
            'Accuracy': accuracy,
            'Macro Precision': precision_macro,
            'Weighted Precision': precision_weighted,
            'Macro Recall': recall_macro,
            'Weighted Recall': recall_weighted,
            'Macro F1': f1_macro,
            'Weighted F1': f1_weighted,

        }
        new_row_df = pd.DataFrame([new_row])
        new_row_df.to_csv("./gaussian_augmenter.csv", mode='a', index=False, header=False)




In [5]:
# def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
#     model.eval()
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model.to(device)
    
#     all_preds = []
#     all_labels = []
#     probabilities = []

#     num_classes = len(np.unique(y_test))
    
#     # X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
#     X_test_tensor , y_test_tensor= preprocess(X_test, y_test, gaussian_augmenter)
#     X_test_tensor = X_test_tensor[:len(X_test)] 
#     y_test_tensor = y_test_tensor[:len(y_test)] 
#     X_test_tensor = X_test_tensor.to(device)
#     y_test_tensor = y_test_tensor.to(device)

#     # X_test_tensor = preprocess(X_test, gaussian_augmenter).to(device)
#     # y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)
    
#     test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
#     test_loader = DataLoader(dataset=test_dataset)

#     with torch.no_grad():
        
#         for inputs, labels in test_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             preds = torch.argmax(outputs, dim=1)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#             probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
#         all_preds = np.array(all_preds)
#         all_labels = np.array(all_labels)
#         probabilities = np.array(probabilities)

#         np.save(f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Defense_Label/WUSTL_Def6/y_pred_{attack_name}{eps}_Def6.npy", all_preds)


In [6]:
x_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
x_train = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_train.npy')
x_val = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_val.npy')
y_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')
y_train = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_train.npy')
y_val = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_val.npy')

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [8]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [9]:
# x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
x_train_tensor , y_train_tensor= preprocess(x_train, y_train, gaussian_augmenter)
# y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
x_train_tensor = x_train_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)

# x_val_tensor = torch.tensor(x_val, dtype=torch.float32).to(device)
x_val_tensor , y_val_tensor= preprocess(x_val,y_val, gaussian_augmenter)
# y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)
x_val_tensor = x_val_tensor.to(device)
y_val_tensor = y_val_tensor.to(device)


train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=True)

In [10]:
class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)

# Compile model
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

# Early stopping variables
min_delta = 0.001
patience = 5
patience_counter = 0
best_loss = float('inf')

In [11]:
# for epoch in range(50):
#     model.train()
#     train_loss = 0.0
#     for inputs, labels in train_loader:
#         inputs, labels = inputs.to(device), labels.to(device)
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = loss_function(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         train_loss += loss.item()

#     avg_train_loss = train_loss / len(train_loader)

#     model.eval()
#     val_train_loss = 0.0
#     correct_predictions = 0
#     with torch.no_grad():
#         for inputs, labels in val_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             loss = loss_function(outputs, labels)
#             val_train_loss += loss.item()
#             _, predicted = torch.max(outputs.data, 1)
#             correct_predictions += (predicted == labels).sum().item()

#     avg_val_loss = val_train_loss / len(val_loader)
#     val_accuracy = correct_predictions / len(val_dataset)

#     print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

#     # Early stopping check using min_delta
#     if best_loss - avg_val_loss > min_delta:
#         best_loss = avg_val_loss
#         patience_counter = 0
#     else:
#         patience_counter += 1

#     if patience_counter >= patience:
#         print("Early stopping triggered")
#         break

In [12]:

model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Defense/preprocessor/dnn_pytorch_gaussian_augmenter.pt"))

/tmp/ipykernel_10867/1125061166.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Defense

<All keys matched successfully>

In [13]:
# calculate_performance_metrics(x_test, y_test, model, 'DNN', 'Baseline', '0')

In [14]:
def print_empty_file():
    new_row = {
        "model" : "0",
        "attack_model" : "0",
        'epsilon': "0",
        'Accuracy': "0",
        'Macro Precision': "0",
        'Weighted Precision': "0",
        'Macro Recall': "0",
        'Weighted Recall': "0",
        'Macro F1': "0",
        'Weighted F1': "0",
    }
    new_row_df = pd.DataFrame([new_row])
    new_row_df.to_csv("./gaussian_augmenter.csv", mode='a', index=False, header=False)

In [15]:
epsilon = 0
Def = "Def6"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start XGB 100 baseline
(238512, 42) (238512,)
Accuracy: 0.9999580733883411

macro
Precision: 0.9652471861195441
Recall: 0.9771591788886799
F1 Score: 0.9710854795261875

weighted
Precision: 0.9999592778255486
Recall: 0.9999580733883411
F1 Score: 0.9999585635141864

start XGB 100 BIM
(2418, 42) (2418,)
Accuracy: 0.7105045492142267

macro
Precision: 0.7111778204840074
Recall: 0.5007458485786659
F1 Score: 0.5392744565806142

weighted
Precision: 0.7472070898016441
Recall: 0.7105045492142267
F1 Score: 0.6837098205789132

start XGB 100 FGSM
(7785, 42) (7785,)
Accuracy: 0.363519588953115

macro
Precision: 0.46869030573444836
Recall: 0.5645123152709359
F1 Score: 0.4651155936356689

weighted
Precision: 0.21254587011044987
Recall: 0.363519588953115
F1 Score: 0.23307816264042408

start XGB 100 PGD
(2418, 42) (2418,)
Accuracy: 0.7105045492142267

macro
Precision: 0.7111778204840074
Recall: 0.5007458485786659
F1 Score: 0.5392744565806142

weighted
Precision: 0.7472070898016441
Recall: 0.710504549214

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.8723542116630669

macro
Precision: 0.5692806603773585
Recall: 0.5336427178861173
F1 Score: 0.5448957718130645

weighted
Precision: 0.9020689641998451
Recall: 0.8723542116630669
F1 Score: 0.8739815194587923

start XGB 100 VNIFGSM
(9599, 42) (9599,)
Accuracy: 0.8512345035941243

macro
Precision: 0.6003299198438314
Recall: 0.6502813583398204
F1 Score: 0.5879052235789859

weighted
Precision: 0.7326077893171431
Recall: 0.8512345035941243
F1 Score: 0.786877985835313

start XGB 50 baseline
(238416, 42) (238416,)
Accuracy: 0.9999538621569022

macro
Precision: 0.9573028447466314
Recall: 0.9866213205165305
F1 Score: 0.9711188842853014

weighted
Precision: 0.9999573250241949
Recall: 0.9999538621569022
F1 Score: 0.9999548127340293

start XGB 50 BIM
(2035, 42) (2035,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9115479115479116

macro
Precision: 0.4296941608245669
Recall: 0.42101472771143217
F1 Score: 0.419467970614384

weighted
Precision: 0.853533691614429
Recall: 0.9115479115479116
F1 Score: 0.880151700443217

start XGB 50 FGSM
(5432, 42) (5432,)
Accuracy: 0.2106038291605302

macro
Precision: 0.38857626363325154
Recall: 0.3428731342302179
F1 Score: 0.20202136773219612

weighted
Precision: 0.752457932454831
Recall: 0.2106038291605302
F1 Score: 0.15386167319590183

start XGB 50 PGD
(2035, 42) (2035,)


...

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9990190926607112

macro
Precision: 0.5998161602234361
Recall: 0.5803508771929825
F1 Score: 0.5895759345249302

weighted
Precision: 0.9990899947160958
Recall: 0.9990190926607112
F1 Score: 0.9990289973833435

start RF 1 SINIFGSM
(2133, 42) (2133,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9995311767463666

macro
Precision: 0.4997655883731833
Recall: 0.5
F1 Score: 0.49988276670574444

weighted
Precision: 0.9990625732879763
Recall: 0.9995311767463666
F1 Score: 0.9992968200812443

start RF 1 VNIFGSM
(3044, 42) (3044,)
Accuracy: 0.20959264126149804

macro
Precision: 0.4423474491497165
Recall: 0.23350027824151365
F1 Score: 0.12737190051646283

weighted
Precision: 0.8332631902267905
Recall: 0.20959264126149804
F1 Score: 0.07477954871836777



/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
epsilon = 0
Def = "Def6"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ExcludeCaFA/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ExcludeCaFA/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_ExcludeCaFA"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start XGB 100 baseline
(238733, 42) (238733,)
Accuracy: 0.9999790561003297

macro
Precision: 0.9824872286079183
Recall: 0.9899867816272725
F1 Score: 0.986005958908532

weighted
Precision: 0.999979736777069
Recall: 0.9999790561003297
F1 Score: 0.9999791786528519

start XGB 100 BIM
(1799, 42) (1799,)
Accuracy: 0.9349638688160089

macro
Precision: 0.7679842326192533
Recall: 0.7486733430013631
F1 Score: 0.7535831645807038

weighted
Precision: 0.9443603866826625
Recall: 0.9349638688160089
F1 Score: 0.9333271812841527

start XGB 100 FGSM
(4778, 42) (4778,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.4616994558392633

macro
Precision: 0.6273782031007002
Recall: 0.5166238733604516
F1 Score: 0.4662071396166847

weighted
Precision: 0.8422340382086131
Recall: 0.4616994558392633
F1 Score: 0.43855147929419924

start XGB 100 PGD
(1799, 42) (1799,)
Accuracy: 0.9349638688160089

macro
Precision: 0.7679842326192533
Recall: 0.7486733430013631
F1 Score: 0.7535831645807038

weighted
Precision: 0.9443603866826625
Recall: 0.9349638688160089
F1 Score: 0.9333271812841527

start XGB 100 DF
(687720, 42) (687720,)
Accuracy: 0.9656211830396091

macro
Precision: 0.4415868832769971
Recall: 0.33596975122023537
F1 Score: 0.34068980647002217

weighted
Precision: 0.9334012963337835
Recall: 0.9656211830396091
F1 Score: 0.9491219531844404

start XGB 100 AutoPGD
(39495, 42) (39495,)
Accuracy: 0.9513356121027978

macro
Precision: 0.7376031973081544
Recall: 0.7829093680497632
F1 Score: 0.7450880166611105

weighted
Precision: 0.9539876509399656
Recall: 0.9513356121027978
F1 Score: 0.9479790408403836

s

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.977013791724965

macro
Precision: 0.7413572415534306
Recall: 0.6922386272944931
F1 Score: 0.6940032519416196

weighted
Precision: 0.9805973774058365
Recall: 0.977013791724965
F1 Score: 0.9765096369373456

start XGB 50 FGSM
(10506, 42) (10506,)
Accuracy: 0.53912050256996

macro
Precision: 0.4761504867808891
Recall: 0.5639913471047017
F1 Score: 0.4178053688930904

weighted
Precision: 0.9054020926867339
Recall: 0.53912050256996
F1 Score: 0.6103941639060085

start XGB 50 PGD
(5003, 42) (5003,)


...

Accuracy: 0.9920773742154543

macro
Precision: 0.7614490371945698
Recall: 0.7202761863188981
F1 Score: 0.7387340341561508

weighted
Precision: 0.9954604731562376
Recall: 0.9920773742154543
F1 Score: 0.9936665322486115

start XGB 10 FGSM
(13573, 42) (13573,)
Accuracy: 0.8655418846238857

macro
Precision: 0.7016462531205874
Recall: 0.6350602512951014
F1 Score: 0.6385414310835914

weighted
Precision: 0.9218365593516791
Recall: 0.8655418846238857
F1 Score: 0.8774283285091247

start XGB 10 PGD
(9719, 42) (9719,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9920773742154543

macro
Precision: 0.7614490371945698
Recall: 0.7202761863188981
F1 Score: 0.7387340341561508

weighted
Precision: 0.9954604731562376
Recall: 0.9920773742154543
F1 Score: 0.9936665322486115

start XGB 10 DF
(687682, 42) (687682,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9655931084425651

macro
Precision: 0.21147070183105865
Recall: 0.2482752596738425
F1 Score: 0.2230019856158932

weighted
Precision: 0.9331888768206913
Recall: 0.9655931084425651
F1 Score: 0.949113160119357

start XGB 10 AutoPGD
(47980, 42) (47980,)
Accuracy: 0.9684035014589413

macro
Precision: 0.6920487338470718
Recall: 0.8309916885508681
F1 Score: 0.7244890079611895

weighted
Precision: 0.973705139401697
Recall: 0.9684035014589413
F1 Score: 0.9704508105171922

start XGB 10 ZOO
(716232, 42) (716232,)


In [ ]:
epsilon = 0
Def = "Def6"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"


total_num_attack = ["2", "3"]
for num_attack in total_num_attack:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    print(x_test_adv.shape, y_test_adv.shape)
    
                    m_per_name = f"{m_name}{p}_Exclude{num_attack}Attack"
                    calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()


start XGB 100 baseline
(238743, 42) (238743,)
Accuracy: 0.9999539253506909

macro
Precision: 0.9689399760881962
Recall: 0.9847404573515348
F1 Score: 0.9752692887924436

weighted
Precision: 0.9999597760233208
Recall: 0.9999539253506909
F1 Score: 0.9999552374349165

start XGB 100 BIM
(474662, 42) (474662,)
Accuracy: 0.999812498156583

macro
Precision: 0.8754236036081726
Recall: 0.9446513706842401
F1 Score: 0.9016275034317782

weighted
Precision: 0.9998390186034571
Recall: 0.999812498156583
F1 Score: 0.9998208712990406

start XGB 100 FGSM
(288975, 42) (288975,)
Accuracy: 0.9374651786486721

macro
Precision: 0.7189503732889356
Recall: 0.6501655680463324
F1 Score: 0.6023377532927591

weighted
Precision: 0.9456512556079771
Recall: 0.9374651786486721
F1 Score: 0.929371649446929

start XGB 100 PGD
(474662, 42) (474662,)
Accuracy: 0.999812498156583

macro
Precision: 0.8754236036081726
Recall: 0.9446513706842401
F1 Score: 0.9016275034317782

weighted
Precision: 0.9998390186034571
Recall: 0.99981

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9999312651789396

macro
Precision: 0.7989926362297497
Recall: 0.7029066171923315
F1 Score: 0.7360630984650522

weighted
Precision: 0.9999542239589003
Recall: 0.9999312651789396
F1 Score: 0.9999355195450857

start XGB 100 PGD
(217706, 42) (217706,)
Accuracy: 0.9999173196880197

macro
Precision: 0.947433904273901
Recall: 0.9315575221238938
F1 Score: 0.9289519831640701

weighted
Precision: 0.9999186480904014
Recall: 0.9999173196880197
F1 Score: 0.99990969368236

start XGB 100 DF
(59, 42) (59,)
Accuracy: 0.2033898305084746

macro
Precision: 0.04528301886792453
Recall: 0.16
F1 Score: 0.07058823529411765

weighted
Precision: 0.057563159577870164
Recall: 0.2033898305084746
F1 Score: 0.0897308075772682

start XGB 100 AutoPGD


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

(233897, 42) (233897,)
Accuracy: 0.9999401445935604

macro
Precision: 0.9903688865320437
Recall: 0.9218085632255315
F1 Score: 0.9492813185551041

weighted
Precision: 0.9999379168459755
Recall: 0.9999401445935604
F1 Score: 0.9999343631394565

start XGB 100 ZOO
(716206, 42) (716206,)
Accuracy: 0.9999790563050296

macro
Precision: 0.9861294370547753
Recall: 0.981534097360473
F1 Score: 0.9838159996418596

weighted
Precision: 0.9999789010509307
Recall: 0.9999790563050296
F1 Score: 0.9999789642003177

start XGB 100 CaFA
(380859, 42) (380859,)
Accuracy: 0.20408077530004542

macro
Precision: 0.2758307938815293
Recall: 0.5785050658107574
F1 Score: 0.20598462850570892

weighted
Precision: 0.7914230719103389
Recall: 0.20408077530004542
F1 Score: 0.28848002823576535

start XGB 100 SINIFGSM
(219759, 42) (219759,)
Accuracy: 0.9999391909590989

macro
Precision: 0.9496743960400599
Recall: 0.9422222222222223
F1 Score: 0.9393308415620369

weighted
Precision: 0.9999442834453875
Recall: 0.9999391909590989

...

Accuracy: 0.9999819607737024

macro
Precision: 0.7297894736842105
Recall: 0.73
F1 Score: 0.7298946925021061

weighted
Precision: 0.9999729487560173
Recall: 0.9999819607737024
F1 Score: 0.9999774528667937

start XGB 20 VNIFGSM
(219983, 42) (219983,)
Accuracy: 0.9999772709709387

macro
Precision: 0.9994772298448427
Recall: 0.9707070707070706
F1 Score: 0.9845001788259408

weighted
Precision: 0.9999773184366039
Recall: 0.9999772709709387
F1 Score: 0.9999767849788798

start XGB 10 baseline
(238752, 42) (238752,)
Accuracy: 0.9999162310682215

macro
Precision: 0.9339635298850684
Recall: 0.9726930881228231
F1 Score: 0.9520048696820715

weighted
Precision: 0.9999253220572111
Recall: 0.9999162310682215
F1 Score: 0.9999192910622896

start XGB 10 BIM
(230726, 42) (230726,)
Accuracy: 0.9998179658989451

macro
Precision: 0.8877937759391991
Recall: 0.9833703068384143
F1 Score: 0.9233417922486534

weighted
Precision: 0.999883647416988
Recall: 0.9998179658989451
F1 Score: 0.9998375193785913

start XGB 

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start XGB 10 AutoPGD
(236027, 42) (236027,)
Accuracy: 0.9997585022052562

macro
Precision: 0.8710489615269358
Recall: 0.9278853468764068
F1 Score: 0.8960069223119815

weighted
Precision: 0.9998004717419888
Recall: 0.9997585022052562
F1 Score: 0.9997733034555792

start XGB 10 ZOO
(716255, 42) (716255,)
Accuracy: 0.9999190232528917

macro
Precision: 0.9359193088062654
Recall: 0.9757932496100814
F1 Score: 0.9545601487857059

weighted
Precision: 0.9999276339818872
Recall: 0.9999190232528917
F1 Score: 0.9999219013867573

start XGB 10 CaFA
(477621, 42) (477621,)
Accuracy: 0.17249241553449282

macro
Precision: 0.2712900908530086
Recall: 0.5560155954082573
F1 Score: 0.19634667684653687

weighted
Precision: 0.7744365368706232
Recall: 0.17249241553449282
F1 Score: 0.2477005266011158

start XGB 10 SINIFGSM
(230678, 42) (230678,)
Accuracy: 0.9998222630679996

macro
Precision: 0.8897414

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9450151057401812

macro
Precision: 0.2356403463250758
Recall: 0.23116703266732186
F1 Score: 0.23263510983328767

weighted
Precision: 0.9183542520308975
Recall: 0.9450151057401812
F1 Score: 0.9314358199128283

start XGB 1 AutoPGD
(242570, 42) (242570,)


In [18]:
epsilon = 0

Def = "Def6"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

rec_list = ["Euclidean", "cosine", "manhattan"]
for rec in rec_list:
    for attack in attack_names:
        print(f"start {attack}")
        
        x_path = f"{base_path}/Recommendation_{rec}/x_test_adv_{attack}_{Def}.npy"
        y_path = f"{base_path}/Recommendation_{rec}/y_test_adv_{attack}_{Def}.npy"
    
        try:
            x_test_adv = np.load(x_path)
            y_test_adv = np.load(y_path)
            m_per_name = f"Recommendation_{rec}"
            calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
        except FileNotFoundError:
            print(x_path, "not found")
            print_empty_file()



start baseline
Accuracy: 0.9999781413120459

macro
Precision: 0.9797949209156105
Recall: 0.9875935658544354
F1 Score: 0.9836073500530584

weighted
Precision: 0.9999786880584105
Recall: 0.9999781413120459
F1 Score: 0.999978321402588

start BIM
Accuracy: 0.9856878276255518

macro
Precision: 0.9391665035095629
Recall: 0.7867632067468155
F1 Score: 0.8443296513440494

weighted
Precision: 0.9858881123526814
Recall: 0.9856878276255518
F1 Score: 0.9850126835010362

start FGSM
Accuracy: 0.9768070888306857

macro
Precision: 0.9520299319052168
Recall: 0.7096516710365444
F1 Score: 0.8001064949088068

weighted
Precision: 0.9773239341057621
Recall: 0.9768070888306857
F1 Score: 0.9752362059727618

start PGD
Accuracy: 0.9856878276255518

macro
Precision: 0.9391665035095629
Recall: 0.7867632067468155
F1 Score: 0.8443296513440494

weighted
Precision: 0.9858881123526814
Recall: 0.9856878276255518
F1 Score: 0.9850126835010362

start DF
Accuracy: 0.9643561452067765

macro
Precision: 0.42303510828198015
Rec

In [ ]:
epsilon = 0
Def = "Def6"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_Cluster/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_Cluster/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_Cluster"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


In [15]:
epsilon = 0
Def = "Def6"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ActiveLearning/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ActiveLearning/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_ActiveLearning"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start XGB 100 baseline
(238512, 42) (238512,)
Accuracy: 0.9999580733883411

macro
Precision: 0.9652471861195441
Recall: 0.9771591788886799
F1 Score: 0.9710854795261875

weighted
Precision: 0.9999592778255486
Recall: 0.9999580733883411
F1 Score: 0.9999585635141864

start XGB 100 BIM
(2418, 42) (2418,)
Accuracy: 0.7105045492142267

macro
Precision: 0.7111778204840074
Recall: 0.5007458485786659
F1 Score: 0.5392744565806142

weighted
Precision: 0.7472070898016441
Recall: 0.7105045492142267
F1 Score: 0.6837098205789132

start XGB 100 FGSM
(7785, 42) (7785,)
Accuracy: 0.363519588953115

macro
Precision: 0.46869030573444836
Recall: 0.5645123152709359
F1 Score: 0.4651155936356689

weighted
Precision: 0.21254587011044987
Recall: 0.363519588953115
F1 Score: 0.23307816264042408

start XGB 100 PGD
(2418, 42) (2418,)
Accuracy: 0.7105045492142267

macro
Precision: 0.7111778204840074
Recall: 0.5007458485786659
F1 Score: 0.5392744565806142

weighted
Precision: 0.7472070898016441
Recall: 0.710504549214

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.8723542116630669

macro
Precision: 0.5692806603773585
Recall: 0.5336427178861173
F1 Score: 0.5448957718130645

weighted
Precision: 0.9020689641998451
Recall: 0.8723542116630669
F1 Score: 0.8739815194587923

start XGB 100 VNIFGSM
(9599, 42) (9599,)
Accuracy: 0.8512345035941243

macro
Precision: 0.6003299198438314
Recall: 0.6502813583398204
F1 Score: 0.5879052235789859

weighted
Precision: 0.7326077893171431
Recall: 0.8512345035941243
F1 Score: 0.786877985835313

start XGB 50 baseline
(238425, 42) (238425,)
Accuracy: 0.9999286987522281

macro
Precision: 0.9434387732012676
Recall: 0.9763096737838556
F1 Score: 0.958595049093401

weighted
Precision: 0.9999362479597673
Recall: 0.9999286987522281
F1 Score: 0.9999311006041318

start XGB 50 BIM
(184163, 42) (184163,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9991692142286995

macro
Precision: 0.653331279752212
Recall: 0.47122846673672536
F1 Score: 0.46226285166139097

weighted
Precision: 0.9993785660196597
Recall: 0.9991692142286995
F1 Score: 0.9990249307697944

start XGB 50 FGSM
(9534, 42) (9534,)
Accuracy: 0.24124187119781834

macro
Precision: 0.38959775399040053
Recall: 0.37024778249585316
F1 Score: 0.23428657848893225

weighted
Precision: 0.6931754815583628
Recall: 0.24124187119781834
F1 Score: 0.12657115952437478

start XGB 50 PGD
(184163, 42) (184163,)


...

Accuracy: 0.9697885196374623

macro
Precision: 0.7331117742810636
Recall: 0.75
F1 Score: 0.7414035087719298

weighted
Precision: 0.9405523963457716
Recall: 0.9697885196374623
F1 Score: 0.9549308316107489

start RF 20 DF
(690045, 42) (690045,)
Accuracy: 0.962369120854437

macro
Precision: 0.419353491149169
Recall: 0.2672430070068347
F1 Score: 0.2791694035401009

weighted
Precision: 0.9272154351323821
Recall: 0.962369120854437
F1 Score: 0.9443684462667594

start RF 20 AutoPGD
(5372, 42) (5372,)
Accuracy: 0.7524199553239017

macro
Precision: 0.6119967562163426
Recall: 0.5895694166421103
F1 Score: 0.58769406202729

weighted
Precision: 0.9419573063614042
Recall: 0.7524199553239017
F1 Score: 0.8361223051386685

start RF 20 ZOO
(675140, 42) (675140,)
Accuracy: 0.9999274224605267

macro
Precision: 0.9434351163508126
Recall: 0.9856182398065275
F1 Score: 0.9622301589816402

weighted
Precision: 0.9999378502894456
Recall: 0.9999274224605267
F1 Score: 0.9999302553671586

start RF 20 CaFA
(58578, 42

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9192825112107623

macro
Precision: 0.45964125560538116
Recall: 0.5
F1 Score: 0.47897196261682246

weighted
Precision: 0.8450803354179653
Recall: 0.9192825112107623
F1 Score: 0.8806210971878798

start RF 20 VNIFGSM
(25486, 42) (25486,)
Accuracy: 0.5916581652672055

macro
Precision: 0.5506135444187927
Recall: 0.5949452795841219
F1 Score: 0.4526650321495113

weighted
Precision: 0.628202579322134
Recall: 0.5916581652672055
F1 Score: 0.4434726298775842

start RF 10 baseline
(224272, 42) (224272,)
Accuracy: 0.9999197403153314

macro
Precision: 0.9375231124807396
Recall: 0.9798598837913086
F1 Score: 0.9559597298388673

weighted
Precision: 0.9999325123367147
Recall: 0.9999197403153314
F1 Score: 0.9999232026756966

start RF 10 BIM
(1092, 42) (1092,)
Accuracy: 0.9963369963369964

macro
Precision: 0.9744955609362389
Recall: 0.9368183367416497
F1 Score: 0.9543647573990318

weighted
Precision: 0.996220143859369
Recall: 0.9963369963369964
F1 Score: 0.9962179287321101

start RF 10 FGSM
(1

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9594405594405594

macro
Precision: 0.5
Recall: 0.4797202797202797
F1 Score: 0.489650249821556

weighted
Precision: 1.0
Recall: 0.9594405594405594
F1 Score: 0.979300499643112

start RF 10 VNIFGSM
(19414, 42) (19414,)
Accuracy: 0.8593283197692387

macro
Precision: 0.4813529207512836
Recall: 0.5298151888969808
F1 Score: 0.4730043693964119

weighted
Precision: 0.7542704114963599
Recall: 0.8593283197692387
F1 Score: 0.7963019951236896

start RF 1 baseline
(223690, 42) (223690,)
Accuracy: 0.99991506102195

macro
Precision: 0.9373986655867947
Recall: 0.9793065992276251
F1 Score: 0.9561481773472241

weighted
Precision: 0.999927100108435
Recall: 0.99991506102195
F1 Score: 0.9999186070704756

start RF 1 BIM
(29510, 42) (29510,)
Accuracy: 0.9605896306336835

macro
Precision: 0.6172286917807634
Recall: 0.6601993183042902
F1 Score: 0.5528288383638997

weighted
Precision: 0.9600990864463625
Recall: 0.9605896306336835
F1 Score: 0.9554599713136718

start RF 1 FGSM
(423669, 42) (423669,)
Ac

In [17]:
epsilon = 0

Def = "Def6"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]
active_learning_name = ["DensityWeighted", "BatchMode"]


base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

for ac_name in active_learning_name:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_ActiveLearning_{ac_name}/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_ActiveLearning_{ac_name}/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    m_per_name = f"{m_name}{p}_ActiveLearning_{ac_name}"
                    calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()



start XGB 100 baseline
Accuracy: 0.9999580733883411

macro
Precision: 0.9652471861195441
Recall: 0.9771591788886799
F1 Score: 0.9710854795261875

weighted
Precision: 0.9999592778255486
Recall: 0.9999580733883411
F1 Score: 0.9999585635141864

start XGB 100 BIM
Accuracy: 0.7105045492142267

macro
Precision: 0.7111778204840074
Recall: 0.5007458485786659
F1 Score: 0.5392744565806142

weighted
Precision: 0.7472070898016441
Recall: 0.7105045492142267
F1 Score: 0.6837098205789132

start XGB 100 FGSM
Accuracy: 0.363519588953115

macro
Precision: 0.46869030573444836
Recall: 0.5645123152709359
F1 Score: 0.4651155936356689

weighted
Precision: 0.21254587011044987
Recall: 0.363519588953115
F1 Score: 0.23307816264042408

start XGB 100 PGD
Accuracy: 0.7105045492142267

macro
Precision: 0.7111778204840074
Recall: 0.5007458485786659
F1 Score: 0.5392744565806142

weighted
Precision: 0.7472070898016441
Recall: 0.7105045492142267
F1 Score: 0.6837098205789132

start XGB 100 DF
Accuracy: 0.9648522008238106

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.8723542116630669

macro
Precision: 0.5692806603773585
Recall: 0.5336427178861173
F1 Score: 0.5448957718130645

weighted
Precision: 0.9020689641998451
Recall: 0.8723542116630669
F1 Score: 0.8739815194587923

start XGB 100 VNIFGSM
Accuracy: 0.8512345035941243

macro
Precision: 0.6003299198438314
Recall: 0.6502813583398204
F1 Score: 0.5879052235789859

weighted
Precision: 0.7326077893171431
Recall: 0.8512345035941243
F1 Score: 0.786877985835313

start XGB 50 baseline
Accuracy: 0.9999412997903564

macro
Precision: 0.9522788985931172
Recall: 0.9766728604973676
F1 Score: 0.9638458813624442

weighted
Precision: 0.9999453490588199
Recall: 0.9999412997903564
F1 Score: 0.9999426505729926

start XGB 50 BIM
Accuracy: 0.9384288747346072

macro
Precision: 0.6265512265512265
Recall: 0.675531914893617
F1 Score: 0.5746888758665254

weighted
Precision: 0.924703510690772
Recall: 0.9384288747346072
F1 Score: 0.9272496413974554

start XGB 50 FGSM
Accuracy: 0.16857532783480686

macro
Precision: 

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9845208285909401

macro
Precision: 0.5
Recall: 0.4922604142954701
F1 Score: 0.4961000229410415

weighted
Precision: 1.0
Recall: 0.9845208285909401
F1 Score: 0.9922000458820831

start XGB 50 VNIFGSM
Accuracy: 0.9058321479374111

macro
Precision: 0.46830172316457375
Recall: 0.6869444056317654
F1 Score: 0.5225175442563692

weighted
Precision: 0.8337301107348115
Recall: 0.9058321479374111
F1 Score: 0.8679415841096815

start XGB 20 baseline


...

Accuracy: 0.9674657534246576

macro
Precision: 0.4837328767123288
Recall: 0.5
F1 Score: 0.4917319408181027

weighted
Precision: 0.9359899840495403
Recall: 0.9674657534246576
F1 Score: 0.9514676252131096

start XGB 1 VNIFGSM
Accuracy: 0.4735557363710334

macro
Precision: 0.5068252959093005
Recall: 0.6307777449822904
F1 Score: 0.5441261051802129

weighted
Precision: 0.2319411083594429
Recall: 0.4735557363710334
F1 Score: 0.3096459324450557

start RF 100 baseline
Accuracy: 0.999978957380278

macro
Precision: 0.9790505675954592
Recall: 0.9943577703468811
F1 Score: 0.9863838728423792

weighted
Precision: 0.9999802777015154
Recall: 0.999978957380278
F1 Score: 0.9999792950798267

start RF 100 BIM
Accuracy: 0.9815384615384616

macro
Precision: 0.7965596330275229
Recall: 0.7
F1 Score: 0.7315982263350684

weighted
Precision: 0.98282874617737
Recall: 0.9815384615384616
F1 Score: 0.979382974200788

start RF 100 FGSM
Accuracy: 0.09869797721460126

macro
Precision: 0.2403942946940089
Recall: 0.28462

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9957191780821918

macro
Precision: 0.5
Recall: 0.4978595890410959
F1 Score: 0.49892749892749894

weighted
Precision: 1.0
Recall: 0.9957191780821918
F1 Score: 0.9978549978549979

start RF 100 VNIFGSM
Accuracy: 0.8830747858300533

macro
Precision: 0.3638924857661917
Recall: 0.5200564153859426
F1 Score: 0.39068093677457083

weighted
Precision: 0.7832942049257311
Recall: 0.8830747858300533
F1 Score: 0.8301143672918266

start RF 50 baseline
Accuracy: 0.9999743859020005

macro
Precision: 0.9781051424401177
Recall: 0.988810117430807
F1 Score: 0.9830740941710255

weighted
Precision: 0.9999756612827304
Recall: 0.9999743859020005
F1 Score: 0.9999746556336174

start RF 50 BIM
Accuracy: 0.7821544949204524

macro
Precision: 0.5218039921329037
Recall: 0.5695092024539877
F1 Score: 0.5422802049622438

weighted
Precision: 0.6141445329285162
Recall: 0.7821544949204524
F1 Score: 0.6879953052618825

start RF 50 FGSM
Accuracy: 0.9527183818417542

macro
Precision: 0.35286809404032093
Recall: 0.2

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9953397637368592

macro
Precision: 0.33198380566801616
Recall: 0.33312778845805074
F1 Score: 0.3325548132457046

weighted
Precision: 0.9919217013613891
Recall: 0.9953397637368592
F1 Score: 0.9936277930389974

start RF 1 VNIFGSM
Accuracy: 0.9570275045680137

macro
Precision: 0.3917024026672143
Recall: 0.39989797075161543
F1 Score: 0.39571442665811607

weighted
Precision: 0.9177915565829132
Recall: 0.9570275045680137
F1 Score: 0.936998957827204



In [ ]:
epsilon = 0

Def = "Def6"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF"]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"
total_num_attack = ["EOAL", "EOAL_ExcludeCaFA", "EOAL_Exclude2Attack", "EOAL_Exclude3Attack"]
for num_attack in total_num_attack:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_ActiveLearning_{num_attack}/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_ActiveLearning_{num_attack}/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    m_per_name = f"{m_name}{p}_ActiveLearning_{num_attack}"
                    calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()



start XGB 100 baseline
Accuracy: 0.9999580733883411

macro
Precision: 0.9652471861195441
Recall: 0.9771591788886799
F1 Score: 0.9710854795261875

weighted
Precision: 0.9999592778255486
Recall: 0.9999580733883411
F1 Score: 0.9999585635141864

start XGB 100 BIM
Accuracy: 0.7105045492142267

macro
Precision: 0.7111778204840074
Recall: 0.5007458485786659
F1 Score: 0.5392744565806142

weighted
Precision: 0.7472070898016441
Recall: 0.7105045492142267
F1 Score: 0.6837098205789132

start XGB 100 FGSM
Accuracy: 0.363519588953115

macro
Precision: 0.46869030573444836
Recall: 0.5645123152709359
F1 Score: 0.4651155936356689

weighted
Precision: 0.21254587011044987
Recall: 0.363519588953115
F1 Score: 0.23307816264042408

start XGB 100 PGD
Accuracy: 0.7105045492142267

macro
Precision: 0.7111778204840074
Recall: 0.5007458485786659
F1 Score: 0.5392744565806142

weighted
Precision: 0.7472070898016441
Recall: 0.7105045492142267
F1 Score: 0.6837098205789132

start XGB 100 DF
Accuracy: 0.9648522008238106

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.8723542116630669

macro
Precision: 0.5692806603773585
Recall: 0.5336427178861173
F1 Score: 0.5448957718130645

weighted
Precision: 0.9020689641998451
Recall: 0.8723542116630669
F1 Score: 0.8739815194587923

start XGB 100 VNIFGSM
Accuracy: 0.8512345035941243

macro
Precision: 0.6003299198438314
Recall: 0.6502813583398204
F1 Score: 0.5879052235789859

weighted
Precision: 0.7326077893171431
Recall: 0.8512345035941243
F1 Score: 0.786877985835313

start XGB 50 baseline
Accuracy: 0.9999580777824824

macro
Precision: 0.9656835331742222
Recall: 0.9771591788886799
F1 Score: 0.9713205525546942

weighted
Precision: 0.9999591253600109
Recall: 0.9999580777824824
F1 Score: 0.9999584961728586

start XGB 50 BIM
Accuracy: 0.9667384541756401

macro
Precision: 0.7213224216582995
Recall: 0.6968050096433778
F1 Score: 0.6355048112757319

weighted
Precision: 0.9637259672719415
Recall: 0.9667384541756401
F1 Score: 0.9592503097527288

start XGB 50 FGSM
Accuracy: 0.24135203164329377

macro
Precision

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.9872049017841052

macro
Precision: 0.6
Recall: 0.5969804618117229
F1 Score: 0.5984787472035794

weighted
Precision: 0.9994593620472157
Recall: 0.9872049017841052
F1 Score: 0.9932855265858007

start XGB 50 VNIFGSM
Accuracy: 0.9723468247830197

macro
Precision: 0.5015330907777349
Recall: 0.4935211928535095
F1 Score: 0.46155127950268027

weighted
Precision: 0.9535208425002977
Recall: 0.9723468247830197
F1 Score: 0.9601874613560961

start XGB 20 baseline
Accuracy: 0.9999622725348245

macro
Precision: 0.9691249097248426
Recall: 0.9771110618080991
F1 Score: 0.9730723428722543

weighted
Precision: 0.999962919132208
Recall: 0.9999622725348245
F1 Score: 0.9999625506215369

start XGB 20 BIM
Accuracy: 0.9981040923517681

macro
Precision: 0.7046292967447803
Recall: 0.5076366432373126
F1 Score: 0.5383590287049683

weighted
Precision: 0.9981165432069342
Recall: 0.9981040923517681
F1 Score: 0.9976934828736969

start XGB 20 FGSM
Accuracy: 0.1899652887346166

macro
Precision: 0.394635029420

...

Accuracy: 0.9876237623762376

macro
Precision: 0.7975
Recall: 0.7032258064516129
F1 Score: 0.7349123511307374

weighted
Precision: 0.9885932343234323
Recall: 0.9876237623762376
F1 Score: 0.9860976964818119

start RF 20 DF
Accuracy: 0.9656351741797766

macro
Precision: 0.4351130647168081
Recall: 0.3695421295305028
F1 Score: 0.3551582566901684

weighted
Precision: 0.9333613352192625
Recall: 0.9656351741797766
F1 Score: 0.9491458553568828

start RF 20 AutoPGD
Accuracy: 0.9780526154876469

macro
Precision: 0.556144115950071
Recall: 0.618622163254547
F1 Score: 0.4998157718297945

weighted
Precision: 0.995204793456749
Recall: 0.9780526154876469
F1 Score: 0.9858974720627457

start RF 20 ZOO
Accuracy: 0.999987370034143

macro
Precision: 0.9911631647536268
Recall: 0.9928376745975644
F1 Score: 0.9918392908542021

weighted
Precision: 0.9999877417838141
Recall: 0.999987370034143
F1 Score: 0.9999874135849406

start RF 20 CaFA
Accuracy: 0.9978880310217083

macro
Precision: 0.6929665514676289
Recall:

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.9786773794808405

macro
Precision: 0.3333333333333333
Recall: 0.32632663575476556
F1 Score: 0.32979277309174215

weighted
Precision: 0.9996909765142151
Recall: 0.9786773794808405
F1 Score: 0.989072578138244

start RF 20 VNIFGSM
Accuracy: 0.9442098837652924

macro
Precision: 0.41584552389690266
Recall: 0.49020948284249
F1 Score: 0.43556199445761495

weighted
Precision: 0.8935141764930409
Recall: 0.9442098837652924
F1 Score: 0.9181319848548211

start RF 10 baseline
Accuracy: 0.9999370746923402

macro
Precision: 0.9521387043189369
Recall: 0.9796919262229794
F1 Score: 0.9635988355299279

weighted
Precision: 0.9999465577258466
Recall: 0.9999370746923402
F1 Score: 0.9999394301863639

start RF 10 BIM
Accuracy: 0.806156194352582

macro
Precision: 0.6643074912891986
Recall: 0.6206896551724138
F1 Score: 0.6158261327968783

weighted
Precision: 0.6541032949032838
Recall: 0.806156194352582
F1 Score: 0.7210543761683088

start RF 10 FGSM
Accuracy: 0.20002029632636492

macro
Precision: 0.3

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.4616994558392633

macro
Precision: 0.6273782031007002
Recall: 0.5166238733604516
F1 Score: 0.4662071396166847

weighted
Precision: 0.8422340382086131
Recall: 0.4616994558392633
F1 Score: 0.43855147929419924

start XGB 100 PGD
Accuracy: 0.9349638688160089

macro
Precision: 0.7679842326192533
Recall: 0.7486733430013631
F1 Score: 0.7535831645807038

weighted
Precision: 0.9443603866826625
Recall: 0.9349638688160089
F1 Score: 0.9333271812841527

start XGB 100 DF


In [19]:
print("a")

a


In [14]:

# torch.save(model.state_dict(), "./dnn_pytorch_gaussian_augmenter.pt")

In [16]:
# import time

# epsilon_values = [0.01, 0.1, 0.2, 0.3]

# start_time = time.time()

# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'BIM', epsilon)

# end_time = time.time()
# result = end_time - start_time
# print(f"Execution Time: {result:.6f} seconds")